In [ ]:
import cv2
import mediapipe as mp
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Load sentiment analysis model
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
sentiment = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

# Set up MediaPipe hand tracking
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.6, min_tracking_confidence=0.6)
draw = mp.solutions.drawing_utils

# Open the webcam
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Webcam not accessible.")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Camera frame not received.")
        break

    frame = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand in results.multi_hand_landmarks:
            draw.draw_landmarks(frame, hand, mp_hands.HAND_CONNECTIONS)

            thumb_tip = hand.landmark[4]
            thumb_base = hand.landmark[2]

            if thumb_tip.y < thumb_base.y:
                gesture = "WANNA"
                text = "I love this!"
                color = (0, 255, 0)
            else:
                gesture = "WILL NOT"
                text = "I hate this!"
                color = (0, 0, 255)

            label = sentiment(text)[0]
            sentiment_label = label["label"]
            sentiment_score = label["score"]

            cv2.putText(frame, gesture, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.5, color, 3)
            cv2.putText(frame, f"{sentiment_label} ({sentiment_score:.2f})", (50, 200),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    cv2.imshow("Hand Gesture & Sentiment", frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
